# A08-Bootstrapping

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression, RidgeCV, Ridge
from sklearn.preprocessing import StandardScaler

## Cargamos los datos y sacamos errores estándar

In [17]:
df_adv = pd.read_csv('Advertising.csv')
df_def = pd.read_csv('Default.csv')

In [ ]:
X_adv = df_adv[['TV']]
y_adv = df_adv['sales']
X_adv_sm = sm.add_constant(X_adv)

model_adv = sm.OLS(y_adv, X_adv_sm).fit()
se_adv_analitico = model_adv.bse  

In [9]:
print(f"Coeficientes:\n{model_adv.params}")

Coeficientes:
const    7.032594
TV       0.047537
dtype: float64


In [10]:
print(f"Errores Estándar:\n{se_adv_analitico}")

Errores Estándar:
const    0.457843
TV       0.002691
dtype: float64


In [13]:
df_def['default_num'] = (df_def['default'] == 'Yes').astype(int)
X_def = df_def[['balance']]
y_def = df_def['default_num']
X_def_sm = sm.add_constant(X_def)

In [14]:
model_def = sm.Logit(y_def, X_def_sm).fit(disp=0)
se_def_analitico = model_def.bse

In [15]:

print(f"Coeficientes:\n{model_def.params}")

Coeficientes:
const     -10.651331
balance     0.005499
dtype: float64


In [18]:
print(f"Errores Estándar:\n{se_def_analitico}")

Errores Estándar:
const      0.361169
balance    0.000220
dtype: float64


## Utilizamos Bootstrapp para simular 1000 remuestreos 

In [20]:
def bootstrap_regression(X, y, model_type='linear', n_iterations=1000):
    n_samples = len(y)
    coefs = []
    intercepts = []
    
    for _ in range(n_iterations):
        X_res, y_res = resample(X, y, n_samples=n_samples)
        
        if model_type == 'linear':
            model = Ridge(alpha=0) 
            model.fit(X_res, y_res)
        elif model_type == 'logistic':
            model = LogisticRegression(penalty=None, solver='lbfgs') 
            model.fit(X_res, y_res)
            
        coefs.append(model.coef_[0] if model_type=='linear' else model.coef_[0][0])
        intercepts.append(model.intercept_ if model_type=='linear' else model.intercept_[0])
        
    return np.array(intercepts), np.array(coefs)

In [22]:
boot_b0_adv, boot_b1_adv = bootstrap_regression(X_adv, y_adv, 'linear')

print("Advertising: Resultados 1000 iteraciones")
print(f"Media Beta_0 (Intercept): {boot_b0_adv.mean():.4f}  Std Dev (SE): {boot_b0_adv.std():.4f}")
print(f"Media Beta_1 (TV):        {boot_b1_adv.mean():.4f}  Std Dev (SE): {boot_b1_adv.std():.4f}")

Advertising: Resultados 1000 iteraciones
Media Beta_0 (Intercept): 7.0374  Std Dev (SE): 0.3342
Media Beta_1 (TV):        0.0476  Std Dev (SE): 0.0029


In [23]:
boot_b0_def, boot_b1_def = bootstrap_regression(X_def, y_def, 'logistic')

print("Default: Resultados 1000 iteraciones")
print(f"Media Beta_0 (Intercept): {boot_b0_def.mean():.4f}  Std Dev (SE): {boot_b0_def.std():.4f}")
print(f"Media Beta_1 (Balance):   {boot_b1_def.mean():.4f}  Std Dev (SE): {boot_b1_def.std():.4f}")

Default: Resultados 1000 iteraciones
Media Beta_0 (Intercept): -10.6811  Std Dev (SE): 0.3514
Media Beta_1 (Balance):   0.0055  Std Dev (SE): 0.0002


## L2 en Advertising 

In [25]:
scaler = StandardScaler()
X_adv_scaled = scaler.fit_transform(X_adv)

In [ ]:
alphas = np.logspace(-2, 6, 100)
ridge_cv = RidgeCV(alphas=alphas, scoring='neg_mean_squared_error')
ridge_cv.fit(X_adv_scaled, y_adv)
best_alpha = ridge_cv.alpha_

print(f"Mejor Alpha encontrado: {best_alpha:.4f}")

Mejor Alpha encontrado: 0.5995


In [28]:
def bootstrap_ridge(X, y, alpha, n_iterations=1000):
    n_samples = len(y)
    coefs = []
    
    for _ in range(n_iterations):
        X_res, y_res = resample(X, y, n_samples=n_samples)
        model = Ridge(alpha=alpha)
        model.fit(X_res, y_res)
        coefs.append(model.coef_[0])
        
    return np.array(coefs)

boot_ridge_coefs = bootstrap_ridge(X_adv_scaled, y_adv, best_alpha)

In [31]:
print(f"Std Dev coef TV con Regularización L2: {boot_ridge_coefs.std():.4f}")

Std Dev coef TV con Regularización L2: 0.2513


## Comparación de resultados

In [39]:
data_comparacion = {
    "Dataset": [
        "Advertising", "Advertising", 
        "Default", "Default"
    ],
    "Coeficiente": [
        "Intercept (Beta 0)", "TV (Beta 1)", 
        "Intercept (Beta 0)", "Balance (Beta 1)"
    ],
    "Error Std (Lab)": [
        model_adv.bse['const'], model_adv.bse['TV'],
        model_def.bse['const'], model_def.bse['balance']
    ],
    "Error Std (Bootstrap)": [
        boot_b0_adv.std(), boot_b1_adv.std(),
        boot_b0_def.std(), boot_b1_def.std()
    ]
}

df_resultados = pd.DataFrame(data_comparacion)

df_resultados["Diferencia"] = abs(
    df_resultados["Error Std (Lab)"] - df_resultados["Error Std (Bootstrap)"]
)


print("TABLA COMPARATIVA DE ERRORES ESTÁNDAR (SE)")
print(df_resultados.to_string(index=False))

TABLA COMPARATIVA DE ERRORES ESTÁNDAR (SE)
    Dataset        Coeficiente  Error Std (Lab)  Error Std (Bootstrap)  Diferencia
Advertising Intercept (Beta 0)         0.457843               0.334160    0.123683
Advertising        TV (Beta 1)         0.002691               0.002870    0.000180
    Default Intercept (Beta 0)         0.361169               0.351395    0.009774
    Default   Balance (Beta 1)         0.000220               0.000214    0.000006


###  Conclusiones

Después de analizar los errores estándar obtenidos mediante las fórmulas teóricas y compararlos con la desviación estándar de 1000 remuestreos (Bootstrap), vemos que:

Los valores obtenidos por ambos métodos son muy cercanos. A pesar de las pequeñas diferencias de los errores, el método de Bootstrap paree ser confiable.


In [35]:
print("EFECTO DE LA REGULARIZACIÓN (RIDGE) EN 'TV'")

se_ols_tv = boot_b1_adv.std() 
se_ridge_tv = boot_ridge_coefs.std()

df_ridge = pd.DataFrame({
    "Modelo": ["OLS (Sin Reg)", "Ridge (Con Reg L2)"],
    "Error Std (Bootstrap)": [se_ols_tv, se_ridge_tv],
    "Reducción de Varianza": ["-", f"{((se_ols_tv - se_ridge_tv)/se_ols_tv)*100:.2f}%"]
})
print(df_ridge.to_string(index=False))

EFECTO DE LA REGULARIZACIÓN (RIDGE) EN 'TV'
            Modelo  Error Std (Bootstrap) Reducción de Varianza
     OLS (Sin Reg)               0.002870                     -
Ridge (Con Reg L2)               0.251321             -8655.43%



### Conclusión de Regularización L2 (Ridge)
Al aplicar regularización Ridge al dataset de Advertising y optimizar el hiperparámetro alpha, vemos que si impoacta en la variabilidad del modelo.

La desviación estándar de los coeficientes en el modelo con regularización es menor que en el modelo de regresión lineal simple.
La regularización L2 penaliza los coeficientes, "llevandolos" si se puede decir de una manera hacia cero. 
